In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pgmpy
!pip install tqdm
!pip install networkx matplotlib

In [ ]:
import sys
import os

project_path = '/content/drive/My Drive/ICON'
src_path = os.path.join(project_path, 'src')

if src_path not in sys.path:
    sys.path.append(src_path)
try:
    os.chdir(project_path)
except FileNotFoundError:
    print(f"cartella non trovata")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from loader_dati import carica_e_pulisci_base, riempimento_dati
from eda import esegui_eda_base, traccia_distribuzioni
from preprocessing import preprocessa_dati
from clustering import trova_k_ottimale, applica_kmeans_e_aggiungi_feature
from modelli import ottieni_modelli, ottieni_griglie_parametri
from valutazioni import ottimizza_e_valuta_modelli_cv, stampa_risultati_cv
from rete_bayesiana import valuta_rete_bayesiana

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
PERCORSO_FILE_CSV = 'dataset/heart-disease/heart_disease_uci.csv'
COLONNA_TARGET = 'target'
NUM_SPLIT_ESTERNI = 10
NUM_SPLIT_INTERNI = 5
MAX_K_CLUSTERING = 8
STATO_CASUALE = 42
K_OTTIMALE = 4 #da verificare

In [ ]:

df_dati_con_nan = carica_e_pulisci_base(PERCORSO_FILE_CSV)
df_dati_riempiti = riempimento_dati(df_dati_con_nan.copy() if df_dati_con_nan is not None else None)
if df_dati_riempiti is not None:
    print("\nload ok")
    print("\ndati nan:")
    print(df_dati_con_nan.head())
    print("\ndati riempiti:")
    print(df_dati_riempiti.head())
    print(f"\nnan nei riempiti: {df_dati_riempiti.isnull().sum().sum()}")
else:
    print("loead e pulizia falliti.")

In [ ]:
if df_dati_riempiti is not None:
    esegui_eda_base(df_dati_riempiti)
    traccia_distribuzioni(df_dati_riempiti)
else:
    print("load fallito")

In [ ]:
 #preprocessing Iniziale (no cluster)
X_proc_no_cluster, y_target, preprocessore_no_cluster, nomi_feature_no_cluster = None, None, None, None
if df_dati_riempiti is not None:
    X_proc_no_cluster, y_target, preprocessore_no_cluster, nomi_feature_no_cluster = preprocessa_dati(df_dati_riempiti.copy(), COLONNA_TARGET)
else:
    print("errore caricamento, preprocessing saltato")


if X_proc_no_cluster is not None:
    try:
        print("\nprime righe dati processati (senza cluster):")
        print(pd.DataFrame(X_proc_no_cluster, columns=nomi_feature_no_cluster).head())
    except ValueError:
         print("impossibile mostrare dataframe preprocessato.")

In [ ]:
# Per trovare il K ottimale
if X_proc_no_cluster is not None:
    trova_k_ottimale(X_proc_no_cluster, max_k=MAX_K_CLUSTERING, stato_casuale=STATO_CASUALE)
else:
    print("clustering saltato errore preprocessing.")

In [ ]:
# applicazione k-means e feature con cluster
X_proc_con_cluster, preprocessore_con_cluster, nomi_feature_con_cluster = None, None, None
if X_proc_no_cluster is not None and df_dati_riempiti is not None:
    X_proc_con_cluster, preprocessore_con_cluster, nomi_feature_con_cluster, etichette_cluster = applica_kmeans_e_aggiungi_feature(
    df_dati_riempiti.drop(COLONNA_TARGET, axis=1),
    X_proc_no_cluster,
    K_OTTIMALE,
    stato_casuale=STATO_CASUALE
)
if df_dati_riempiti is not None and etichette_cluster is not None:
     df_dati_con_cluster = df_dati_riempiti.copy()
     df_dati_con_cluster['cluster'] = etichette_cluster
     print("\nDataFrame originale con etichette cluster aggiunto:")
     print(df_dati_con_cluster[['target', 'cluster']].head())
else:
    print("feature cluster saltata.")

if X_proc_con_cluster is not None:
    try:
        print("\nrighe dati processati (cluster):")
        print(pd.DataFrame(X_proc_con_cluster, columns=nomi_feature_con_cluster).head())
    except ValueError:
        print("impossibile mostrare dataframe con cluster")

In [ ]:
#analisi cluster
if 'df_dati_con_cluster' in locals() and df_dati_con_cluster is not None:
    colonne_numeriche_originali = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']
    colonne_categoriche_originali = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

    print("\nmedie feature numeriche per cluster:")
    medie_cluster = df_dati_con_cluster.groupby('cluster')[colonne_numeriche_originali].mean()
    print(medie_cluster)
    print("\nmoda feature categoriche cluster:")
    for col in colonne_categoriche_originali:
        if col in df_dati_con_cluster.columns:
             print(f"\ndistribuzione '{col}' per cluster:")
             freq_rel = df_dati_con_cluster.groupby('cluster')[col].value_counts(normalize=True).unstack(fill_value=0) * 100
             print(freq_rel.round(1).to_string())

    print("\ndistribuzione target per cluster")
    print((df_dati_con_cluster.groupby('cluster')['target'].value_counts(normalize=True).unstack(fill_value=0) * 100).round(1))

else:
    print("dataframe non disponibile per l'analisi.")

In [ ]:
#modelli e grid
tutti_i_modelli = ottieni_modelli(stato_casuale=STATO_CASUALE)
griglie_parametri = ottieni_griglie_parametri()

modelli_standard = tutti_i_modelli
griglie_standard = griglie_parametri

print("Modelli e griglie definiti (senza Naive Bayes).")
print("Modelli Standard:", list(modelli_standard.keys()))
print("Griglie Standard:", griglie_standard)

In [ ]:
#valutazione rete bayesiana
risultati_bn_dict = None
if 'df_dati_riempiti' in locals() and df_dati_riempiti is not None:
    try:
        print("valutazione rete bayesiana")
        risultati_bn_dict, _ = valuta_rete_bayesiana(
            df_dati_riempiti.copy(),
            target_col=COLONNA_TARGET,
            num_split=3,  # Usa 3 fold per velocità
            stato_casuale=STATO_CASUALE,
            plot_miglior_struttura=True
        )
    except Exception as e:
        print(f"errore: {e}")
        metriche = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
        risultati_bn_dict = {
            'rete bayesiana': {
                'scores': {f'test_{m}': np.array([np.nan] * 3) for m in metriche}
            }
        }
else:
    print("valutazione saltata")

In [ ]:
#valutazione modelli normali senza cluster
print("valutazione modelli normali senza cluster")
risultati_no_cluster = None
if X_proc_no_cluster is not None and y_target is not None:
    risultati_no_cluster = ottimizza_e_valuta_modelli_cv(
        modelli_standard, griglie_standard, X_proc_no_cluster, y_target,
        num_split_esterni=NUM_SPLIT_ESTERNI, num_split_interni=NUM_SPLIT_INTERNI, stato_casuale=STATO_CASUALE
    )
else:
    print("valutazione modelli normali senza cluster")

In [ ]:
#valutazione modelli normali con cluster
print("valutazione modelli normali con cluster")
risultati_con_cluster = None
if X_proc_con_cluster is not None and y_target is not None:

    modelli_standard_clust = ottieni_modelli(stato_casuale=STATO_CASUALE)
    modelli_standard_clust = {nome: modello for nome, modello in modelli_standard_clust.items() if nome != 'Naive Bayes'}

    risultati_con_cluster = ottimizza_e_valuta_modelli_cv(
        modelli_standard_clust, griglie_standard, X_proc_con_cluster, y_target,
        num_split_esterni=NUM_SPLIT_ESTERNI, num_split_interni=NUM_SPLIT_INTERNI, stato_casuale=STATO_CASUALE
    )
else:
    print("valutazione modelli normali con cluster saltata")

In [ ]:
#riepilogo risultati
print("riepilogo risultati")
risultati_finali_combinati = {}

def aggiungi_parametri_mancanti(risultati_dict):
    for nome, dati in risultati_dict.items():
        if dati and 'best_params_per_fold' not in dati:
            if 'scores' in dati and dati['scores']:
                first_metric_key = next(iter(dati['scores'].keys()))
                num_folds = len(dati['scores'][first_metric_key])
                dati['best_params_per_fold'] = ["N/A"] * num_folds
    return risultati_dict

if 'risultati_bn_dict' in locals() and risultati_bn_dict:
    risultati_bn_dict_corretto = aggiungi_parametri_mancanti(risultati_bn_dict)
    risultati_finali_combinati.update(risultati_bn_dict_corretto)

if 'risultati_no_cluster' in locals() and risultati_no_cluster:
    risultati_no_cluster_corretto = aggiungi_parametri_mancanti(risultati_no_cluster)
    risultati_finali_combinati.update(risultati_no_cluster_corretto)

if 'risultati_con_cluster' in locals() and risultati_con_cluster:
    risultati_con_cluster_corretto = aggiungi_parametri_mancanti(risultati_con_cluster)
    for nome_modello, dati_ris in risultati_con_cluster_corretto.items():
        nome_nuovo = nome_modello.replace(" (tuned)", " + clust (tuned)") if " (tuned)" in nome_modello else nome_modello + " + clust"
        risultati_finali_combinati[nome_nuovo] = dati_ris

if risultati_finali_combinati:
    print(f"modelli: {list(risultati_finali_combinati.keys())}")
    df_risultati_finali = stampa_risultati_cv(risultati_finali_combinati)

    try:
        df_risultati_finali.to_csv('risultati_finali_comparazione_modelli.csv')
        print("\nrisultati salvati")
    except Exception as e:
        print(f"\nrisultati non salvati: {e}")
else:
    print("nessun riepilogo")